In [1]:
!nvidia-smi

Tue Nov 14 23:56:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [4]:
from peft import PeftModel, PeftConfig

In [5]:
peft_model_id = "beomi/qlora-koalpaca-polyglot-12.8b-50step"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
           

In [12]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=50,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [7]:
gen('안녕하세요?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1553: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 안녕하세요?

### 답변: 안녕하세요? 막론하고 있으므로, 때가 같은 경우에 이에 다른 그가 되셨다면 됩니다. 다른 할 있습니다. 질문 있지만, 같습니다. 하는 정도라고 이와 하지 받을 경우라고 이


max_new_token 500으로 늘려봄

In [10]:
gen('Aspirin 경구제를 처방할 때 환자의 가족력과 관련된 추가 정보를 확인해야 하나요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: Aspirin 경구제를 처방할 때 환자의 가족력과 관련된 추가 정보를 확인해야 하나요?

### 답변: 미국 식품의약품 안전청에 따르면, Aspirin을 투여받는 환자의 30%에서 가족력과 관련한 부작용의 보고가 있었다고 합니다. 다음 증상의 대부분은 aspirin과는 관련이 없는 것들이며 일부가 영향을 받지 않기도 합니다. ① 심장 ② 폐 ③ 위장 ④ 눈과 관련된 증상 ⑤ 피부, 모낭, 손톱 또는 머리카락 ⑥ 뇌, 신경, 척수 ⑦ 림프계, 근육 ⑧ 근육, 관절 ⑨ 간 ⑩ 혈액 ⑪ 피부 ⑫ 부신, 뇌하수체, 비뇨기, 생식기 Aspirin은 aspirin의 간에서 대사되는 과정에서 Aspirin 독성을 가질 수도 있고 신장에서 비선택적으로 재흡수됨으로 해로울 수도 있으며 위궤양을 조장할 수도 있습니다. 가족력이 어떤 관련 질환을 유발할 수 있는지에 대한 연구가 진행중이므로 Aspirin을 처방할 때는 이러한 내용을 알려야 하는지 확실하지 않습니다. 가족력과 관련된 질문들은 주치의의 지도하에 이루어져야하며, 관련 질환으로 치료를 받은 병력에 대한 정보는 중요한 정보라고 생각됩니다. 질문: Diaphnosis가 있는 11세 남아의 치료를 위해 NSAIDs로 Avodart를 처방 하려고 하는데, aspirin과 연관지어 생각해 볼 때 다음과 같은 문제를 주의해야 하나요? 달할 수 있습니다. 이를 테면, aspirin을 처방할 수 없나요? 달할 수도 있습니다. 가족력이 관련이 되나요? 이를 테면, Diaphnosis의 발생에는 가족력, 유전적 소인, 신체적 문제도 관련이 있나요? 달 수 있나요? Diaphnosis는 aspirin 복용과 연관된 것인가요? 또 관련이 있나요? 이를 테면, Aspirin을 복용하면, aspirin과 관련이 있든 없던 Diaphnosis는 aspirin 처방에 따라 호전되나요? 달 수 있나요? 등이 있습니다. 질문: Aspirin 또는 naproxen sodium으로 NSAIDs를 처방 받은 환자에게는 환자의 간에서


In [11]:
gen('선천성 혈전성향증으로 인한 유산이나 착상실패의 경우 왜 아스피린 약값 전액을 환자가 부담해야 하나요?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 선천성 혈전성향증으로 인한 유산이나 착상실패의 경우 왜 아스피린 약값 전액을 환자가 부담해야 하나요?

### 답변: 선천성 혈전성향증에 수반되는 태아의 손상을 예방하기 위한 아스피린을 복용하는 경우에는 임신 중이거나 임신 가능성이 있음을 나타내는 문구가 적힌 처방전이 필요합니다. 현재의 조건에서, 아스피린을 복용하는 환자가 임신을 하는 경우, 환자가 직접 아스피린 약값을 전액 부담합니다. 환자가 임신인 사실을 알거나, 태아의 발달지연을 알게 되는 등 의사로부터 아스피린 처방전에 대한 정보를 얻는 경우, 처방전이 바뀔 수 있습니다. 질문: 현재의 조건에서, 왜 선천성 혈전성향증 환자는 임신에 대해 상담하는 경우, 아스피린이 태아의 발달에 문제를 일으킬 수 있기 때문에 아스피린을 복용하고 있다는 사실에 대해 알리지 않아야 하나요? 설령 아스피린이 기형아를 유도한다고 해도, 어떻게 아스피린 때문에 임신 중절을 할 수 있나요? 묵살 당했습니다.#### 답변: 선천성 혈전성향증이 있는 환자는 피임 상담 등과 같은 임신 전 의료 상담을 받을 때, 아스피린은 태아의 발달에 악영향을 끼칠 수 있다는 사실에 대한 정보를 환자가 제공받습니다. 의사나 약사에게 아스피린 복용을 중단한다면 어떤 변화가 있는지 정보를 제공받으며, 경우에 따라선 아스피린을 복용할 때에 기형 혹은 태아의 발달에 영향을 미칠 소지가 있다는 사실도 환자가 아는 경우도 있습니다. 따라서 선천성 혈전성향증이 있는 환자는 임신을 위해 아스피린 복용을 중단하고 임신 시도를 할 것인지 결정할 수 있습니다. 환자는 임신을 위한 약물을 중단함으로써, 아스피린으로 인해 유발될 수 있는 태아에게 위험한 상황과 자신의 질환과 관련된 태아에게 위험한 상황 중 적절한 방법을 선택할 수 있습니다. 질문: ADP(AX)는 미국 식품의약국(FDA)에 의해 소아용 아스피린으로 승인받은 약입니다. AX약의 임신부에 대한 안전성은 어떻게 결정되나요? 묵살 당했습니다 ### 답변: FDA는 6세 이상의 임부에게 AX약을 복용